In [1]:
from pangolin import Given, d, t, I, IID, vmap, sample, E, P, var, std, cov, corr, jags_code
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{:6.2f}'.format}) # print nicely

In [2]:
# you can "vectorize" constants

x = 2
y = IID(x,3)
print('E(y)',E(y))

E(y) [  2.00   2.00   2.00]


In [3]:
# very simple vectorization

def f(a,b):
    return d.norm(a*b,1)

A = np.array([1.,2.])
B = np.array([3.,4.])

x= vmap(f)(A,B)
print('E(x)    ',E(x))
expected = A*B
print('expected',expected)

E(x)     [  3.08   8.04]
expected [  3.00   8.00]


In [4]:
def f(a,b):
    u = a*b
    #s = a+b
    return [d.norm(a,b),d.norm(u,1)]

A = np.array([1.,2.])
B = np.array([3.,4.])

x,y = vmap(f)(A,B)
print('E(x)    ',E(x))
expected = A
print('expected',expected)
print('E(y)    ',E(y))
expected = np.array(A)*np.array(B)
print('expected',expected)

E(x)     [  1.05   2.02]
expected [  1.00   2.00]
E(y)     [  2.97   8.04]
expected [  3.00   8.00]


In [5]:
# Tracing that uses IID
def f(a,b):
    x = a*b
    y = a+b
    return [d.norm(x,y)]

A = IID(d.exp(1),3)
B = [.5,1.5,2.5]
#x,=RV.trace(f,A,B)
x, = vmap(f)(A,B)
print('traced version')
print(E(x))

print('untraced version')
x0 = A[0]*B[0]
y0 = A[0]+B[0]
x1 = A[1]*B[1]
y1 = A[1]+B[1]
x0 = d.norm(x0,y0)
x1 = d.norm(x1,y1)
print(E([x0,x1]))

traced version
[  0.54   1.42   2.55]
untraced version
[0.5466128421399999, 1.47939783778]


In [6]:
# here's a realistic-ish example: you might want to create a normal
# parameterized by location and scale

def norm_loc_scale(loc,scale):
    return d.norm(loc,1/scale**2)

vec_norm_loc_scale = vmap(norm_loc_scale)

x = vec_norm_loc_scale([1,2,3],[4,5,6])
print('E(x)  ',E(x))
print('std(x)',std(x))

E(x)   [  0.98   2.27   3.01]
std(x) [  3.98   4.78   6.15]


In [7]:
# vmap and indexing
x_train = [1,2]

def get_yi(xi,w):
    return xi*w[0]

# vectorized_version
get_y = vmap(get_yi,[True,False])

#w = RV.Constant([3,3,3])
w = [3,4,5]
y = get_y(x_train,w)
E(y)

array([  3.00,   6.00])

In [8]:
# a vmap still works even if one variable flows straight to output
def f(a,b):
    return [d.norm(a,1),d.norm(b,1)]

E(vmap(f,[True,False])([1,2],3))

[array([  0.98,   2.04]), array([  3.01,   3.03])]

In [9]:
# vectorization works even if some things are constants
# or are even constants
def f(a):
    return [d.norm(a,1),a,1] #,a

x = vmap(f)([2,3])
print('E(x)  ',E(x))
print('var(x)',var(x))

E(x)   [array([  1.97,   3.04]), array([  0.00,   0.00]), array([  1.00,   1.00])]
var(x) [array([  1.00,   0.96]), array([  0.00,   0.00]), array([  0.00,   0.00])]


In [10]:
# vmap with indexing

def f(a,b):
    return a[0]*b

A = np.random.randn(5,2)
B = np.random.randn(5,3)
x = vmap(f)(A,B)
print('E(x)')
print(E(x))
print('expected')
print(A[:,0,None]*B)

E(x)
[[  1.09  -3.06  -1.41]
 [ -1.92   0.24  -0.23]
 [  0.17  -1.99  -0.45]
 [  0.01   0.47  -1.48]
 [ -2.94   1.40  -0.04]]
expected
[[  1.09  -3.06  -1.41]
 [ -1.92   0.24  -0.23]
 [  0.17  -1.99  -0.45]
 [  0.01   0.47  -1.48]
 [ -2.94   1.40  -0.04]]
